# Dimensión cliente

In [30]:
from datetime import date
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import holidays
import yaml

### Conexión con la base de datos

In [31]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [32]:
dimension_cliente = pd.read_sql_table('cliente', oltp_conn)
table_ciudad = pd.read_sql_table('ciudad', oltp_conn)
table_departamento = pd.read_sql_table('departamento', oltp_conn)
table_tipo_cliente = pd.read_sql_table('tipo_cliente', oltp_conn)

table_ciudad.head()

,ciudad_id,nombre,departamento_id
0,6,BUGA,1
1,5,BOGOTA,2
2,4,PASTO,4
3,3,POPAYAN,3
4,2,PALMIRA,1


### Módulo de transformación

In [33]:
table_departamento = table_departamento.rename(columns={'nombre' : 'departamento'})
table_ciudad = (pd.merge(table_ciudad.rename(columns={'nombre' : 'ciudad'}), table_departamento, how='inner')
  .drop(columns=['departamento_id']))
table_tipo_cliente = (table_tipo_cliente.drop(columns=['descripcion']).rename(columns={'nombre': 'tipo_cliente'}))

dimension_cliente = pd.merge(dimension_cliente, table_ciudad, how='inner')
dimension_cliente = (pd.merge(dimension_cliente, table_tipo_cliente, how='inner')
  .drop(columns=['ciudad_id', 'tipo_cliente_id', 'coordinador_id', 'activo']))

dimension_cliente.head()

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,sector,ciudad,departamento,tipo_cliente
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,S,CALI,VALLE DEL CAUCA,Persona Juridica
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,industrial,CALI,VALLE DEL CAUCA,Persona Juridica
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,salud,CALI,VALLE DEL CAUCA,Persona Juridica
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,salud,CALI,VALLE DEL CAUCA,Persona Juridica
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,salud,CALI,VALLE DEL CAUCA,Persona Juridica


### Módulo de carga a la bodega de datos

In [34]:
dimension_cliente.set_index('cliente_id', inplace=True)
try:
  dimension_cliente.to_sql('dim_cliente', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")